In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
import pymc3 as pm
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
dowStocks = pd.read_csv(Path('../data/external/dow_jones_index.data'))
dowStocks['date'] = pd.to_datetime(dowStocks['date'])
dowStocks.head()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,2011-01-07,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,2011-01-14,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,2011-01-21,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,2011-01-28,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2011-02-04,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029


In [51]:
dowStocks.dtypes

quarter                                        int64
stock                                         object
date                                  datetime64[ns]
open                                          object
high                                          object
low                                           object
close                                         object
volume                                         int64
percent_change_price                         float64
percent_change_volume_over_last_wk           float64
previous_weeks_volume                        float64
next_weeks_open                               object
next_weeks_close                              object
percent_change_next_weeks_price              float64
days_to_next_dividend                          int64
percent_return_next_dividend                 float64
dtype: object

In [56]:
def stripDollars(df):
    df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].str.strip('$').astype('float64')
    return df
## TODO: Fix this. 

In [55]:
dowStocks = stripDollars(dowStocks)

AttributeError: 'DataFrame' object has no attribute 'strip'

In [26]:
print('Stocks in DOWJONES')
for stock in dowStocks['stock'].unique():
    print(f'{stock}')


Stocks in DOWJONES
AA
AXP
BA
BAC
CAT
CSCO
CVX
DD
DIS
GE
HD
HPQ
IBM
INTC
JNJ
JPM
KRFT
KO
MCD
MMM
MRK
MSFT
PFE
PG
T
TRV
UTX
VZ
WMT
XOM


In [31]:
print('Dates in dataset:')
print(f'Start date: {dowStocks["date"].min()}')
print(f'End date: {dowStocks["date"].max()}')

Dates in dataset:
Start date: 2011-01-07 00:00:00
End date: 2011-06-24 00:00:00


Note: Original dataset recommends using Q1 as train and Q2 as test


In [42]:
dowStocks[dowStocks['stock'] == 'AA'][['date', 'close']].set_index(['date'])

,close
date,
2011-01-07,$16.42
2011-01-14,$15.97
2011-01-21,$15.79
2011-01-28,$16.13
2011-02-04,$17.14
2011-02-11,$17.37
2011-02-18,$17.28
2011-02-25,$16.68
2011-03-04,$16.58
